# Домашнее задание - 2(1)

Устанавливаем модули и анализаторы

In [1]:
from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
from string import punctuation
import json
from pprint import pprint
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.collocations import *
import os
os.environ["MYSTEM_BIN"] = "C:\\Users\\staro\\mystem.exe"

### 1. Берём текст "Письма к незнакомке"

In [2]:
m = Mystem()
text = open('book.txt', 'r', encoding='utf-8').read()
for word in text.split():
    word.lower().strip(punctuation)  # всё со строчной, без пунктуации

### 2. Обрабатываем книгу с помощью Mystem

Парсим и замеряем время

In [3]:
%%time
m = Mystem()
an = m.analyze(text)  # парсим
for word in an[:10]:
    print(word)

{'analysis': [{'lex': 'письмо', 'wt': 1, 'gr': 'S,сред,неод=(вин,мн|род,ед|им,мн)'}], 'text': 'Письма'}
{'text': ' '}
{'analysis': [{'lex': 'незнакомка', 'wt': 1, 'gr': 'S,жен,од=(пр,ед|дат,ед)'}], 'text': 'незнакомке'}
{'text': '\n'}
{'analysis': [{'lex': 'андре', 'wt': 0.9836665988, 'gr': 'S,имя,муж,од=(пр,мн|пр,ед|вин,мн|вин,ед|дат,мн|дат,ед|род,мн|род,ед|твор,мн|твор,ед|им,мн|им,ед)'}], 'text': 'Андре'}
{'text': ' '}
{'analysis': [{'lex': 'моруа', 'wt': 1, 'gr': 'S,фам,мж,од=(пр,мн|пр,ед|вин,мн|вин,ед|дат,мн|дат,ед|род,мн|род,ед|твор,мн|твор,ед|им,мн|им,ед)'}], 'text': 'Моруа'}
{'text': '\n'}
{'analysis': [{'lex': 'письмо', 'wt': 1, 'gr': 'S,сред,неод=(вин,мн|род,ед|им,мн)'}], 'text': 'Письма'}
{'text': ' '}
Wall time: 11min 40s


Записываем результат в json

In [4]:
with open('data1.json', 'w', encoding='utf-8') as k:
    json.dump(an, k, ensure_ascii=False, indent=4)

### 3. Обрабатываем книгу с помощью Pymorphy

Токенизируем с помощью nltk

In [5]:
sw = stopwords.words('russian')
words = [w.lower() for w in word_tokenize(text) if w.isalpha()]
filtered = [w for w in words if w not in sw]
print(filtered)  # очищенные слова 

['письма', 'незнакомке', 'андре', 'моруа', 'письма', 'незнакомке', 'существуете', 'вместе', 'друг', 'предложил', 'писать', 'неделю', 'мысленно', 'нарисовал', 'ваш', 'образ', 'создал', 'прекрасной', 'лицом', 'разумом', 'знал', 'замедлите', 'возникнуть', 'живой', 'грез', 'моих', 'станете', 'читать', 'мои', 'послания', 'отвечать', 'говорить', 'жаждет', 'услышать', 'автор', 'первого', 'дня', 'придал', 'определенный', 'облик', 'облик', 'редкостно', 'красивой', 'юной', 'женщины', 'которую', 'увидал', 'театре', 'сцене', 'зале', 'никто', 'тех', 'мною', 'рядом', 'знал', 'тех', 'пор', 'обрели', 'глаза', 'губы', 'голос', 'стать', 'подобает', 'остались', 'незнакомкой', 'печати', 'появились', 'моих', 'письма', 'ожидал', 'стал', 'получать', 'ответы', 'лицо', 'собирательное', 'разных', 'незнакомок', 'одна', 'наивная', 'другая', 'вздорная', 'третья', 'шалунья', 'насмешница', 'терпелось', 'затеять', 'вами', 'переписку', 'однако', 'удержался', 'надлежало', 'оставаться', 'всеми', 'стали', 'одной', 'укоря

Разбираем слова с помощью Pymorphy и замеряем время 

In [6]:
%%time
morph = MorphAnalyzer()
slova = []  # список разборов

for word in text:
        parser = morph.parse(word)
        slova.append(parser)

Wall time: 1min 17s


Составляем словарь с леммой и частью речи каждого слова

In [7]:
text = open('book.txt', 'r', encoding='utf-8').read()
text = [word.lower().strip(punctuation) for word in text.split()]
text = [word for word in text if word != '']
part_speech = []  # список частей речи
lemmas = []  # список лемм
razbor = {}  # словарь из лемм и частей речи
for word in text:
    lemmas.append(morph.parse(word)[0].normal_form)
    part_speech.append(morph.parse(word)[0].tag.POS)
razbor = dict(zip(lemmas, part_speech))

Сохраняем разбор в json

In [8]:
with open('data2.json', 'w', encoding='utf-8') as f:
    json.dump(razbor, f, ensure_ascii=False, indent=4)

### 4. Ответим на вопросы

#### Какую долю слов (в процентах) составляет каждая часть речи?

Pymorphy

In [9]:
counter = Counter(part_speech)  # считаем части речи
for key, count in counter.most_common(20):
    a = int(count)/len(part_speech)*100  # частотность
    print(key, a)

NOUN 24.370557764124804
ADJF 11.914227201541983
VERB 10.902300927599084
CONJ 10.793880255390917
PREP 9.01096253463438
NPRO 8.73388748343573
ADVB 5.649921696181183
PRCL 5.3969401276954585
None 5.083724852427418
INFN 3.409227803879051
ADJS 1.3010480664980122
PRTF 0.9998795325864354
PRED 0.5782435851102277
COMP 0.48186965425852313
GRND 0.4577761715455969
NUMR 0.4577761715455969
PRTS 0.30116853391157694
INTJ 0.15660763763402


Mystem (заодно начнём писать код для ответа на следующий вопрос)

In [10]:
lom = []  # список частей речи
adv = []  # список наречий
verb = []  # список глаголов
for worrd in an:
    if 'analysis' in worrd:
        gr = worrd['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]  # вынимаем части речи
        lom.append(pos)  # добавляем в части речи
        if pos == 'ADV':
            adv.append(worrd['text'].lower())
        if pos == 'V':
            verb.append(worrd['text'].lower())

IndexError: list index out of range

In [11]:
cou = Counter(lom)  # считаем количество каждой части речи
for key, count in cou.most_common(20):
    b = int(count)/len(lom)*100
    print(key, b)

S 23.223880597014926
V 17.611940298507463
SPRO 12.477611940298509
CONJ 10.208955223880597
PR 9.194029850746269
A 7.641791044776119
APRO 6.447761194029851
ADV 5.970149253731343
PART 4.955223880597015
ADVPRO 1.3134328358208955
NUM 0.5970149253731344
ANUM 0.1791044776119403
INTJ 0.1791044776119403


#### Топ-20 наречий

In [12]:
r = Counter(adv)  # считаем наречия
for word, count in r.most_common(20):
    print(word, count)

еще 5
можно 4
нужно 3
особенно 3
легко 2
рядом 2
разумней 2
разумеется 2
весьма 2
быстро 2
жаль 2
нет 2
более 2
вполне 2
впереди 2
больше 2
по-прежнему 2
нельзя 2
слишком 1
навыворот 1


#### Топ-20 глаголов

In [13]:
j = Counter(verb)  # считаем глаголы
for word, count in j.most_common(20):
    print(word, count)

говорить 4
быть 4
занимает 4
было 3
прощайте 3
дать 3
знал 3
стать 3
будем 2
надоест 2
любить 2
говорит 2
заподозрить 2
будет 2
ответить 2
понравиться 2
был 2
порождает 2
уклоняться 2
говорил 2


### 5. Биграммы и триграммы

#### Топ-25 биграмм

In [14]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(filtered)
finder.nbest(bigram_measures.likelihood_ratio, 25)  # ищем самые частотные 25 биграмм

[('крайней', 'мере'),
 ('дорогой', 'мэтр'),
 ('конце', 'концов'),
 ('наши', 'дни'),
 ('рода', 'человеческого'),
 ('души', 'моей'),
 ('свой', 'черед'),
 ('никакого', 'значения'),
 ('подобно', 'тому'),
 ('заподозрить', 'разумней'),
 ('клод', 'мориак'),
 ('нынче', 'вечером'),
 ('первые', 'шаги'),
 ('плакучих', 'ив'),
 ('споров', 'предметах'),
 ('средневековых', 'трактатов'),
 ('эпохи', 'возрождения'),
 ('образ', 'жизни'),
 ('правила', 'игры'),
 ('беда', 'непоправима'),
 ('казаться', 'смешным'),
 ('каким', 'образом'),
 ('образом', 'возникла'),
 ('быстро', 'надоедает'),
 ('дать', 'заподозрить')]

#### Топ-25 триграмм

In [15]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
findr = TrigramCollocationFinder.from_words(filtered)
findr.nbest(trigram_measures.likelihood_ratio, 25)  # ищем самые частотные 25 триграмм

[('знаком', 'крайней', 'мере'),
 ('крайней', 'мере', 'давать'),
 ('надеюсь', 'крайней', 'мере'),
 ('остерегайтесь', 'крайней', 'мере'),
 ('согласитесь', 'крайней', 'мере'),
 ('здравствуйте', 'дорогой', 'мэтр'),
 ('расстраивать', 'дорогой', 'мэтр'),
 ('крайней', 'мере', 'сцена'),
 ('крайней', 'мере', 'ваш'),
 ('незнакомка', 'души', 'моей'),
 ('каким', 'образом', 'возникла'),
 ('дорогой', 'мэтр', 'господин'),
 ('дорогой', 'мэтр', 'месте'),
 ('дорогой', 'мэтр', 'наших'),
 ('огорчен', 'дорогой', 'мэтр'),
 ('конце', 'концов', 'исчерпать'),
 ('конце', 'концов', 'перестанет'),
 ('продолжаться', 'конце', 'концов'),
 ('рискует', 'конце', 'концов'),
 ('дать', 'заподозрить', 'разумней'),
 ('крайней', 'мере', 'это'),
 ('имеет', 'никакого', 'значения'),
 ('дорогой', 'мэтр', 'нам'),
 ('тому', 'дорогой', 'мэтр'),
 ('конце', 'концов', 'выходит')]

Заметим, что триграммы - это те же частотные биграммы + ещё слово